In [21]:
import yaml
import pandas as pd
from ehrmonize import EHRmonize

In [ ]:
def accuracy_score(y_true, y_pred):
    return (y_true == y_pred).mean()

In [ ]:
with open('experiments/configs/get_generic_name.yaml', 'r') as file:
    for key, value in yaml.safe_load(file).items():
        globals()[key] = value


for db, path, drugn, gener in zip(source_db, source_csv, input_col_name, gt_col_name):
    data = pd.read_csv(source_csv)
    
    # different engines
    for m in model_id:
        for t in temperature:
            ehrm = EHRmonize(model_id=m, temperature=t)

            # different prompting strategies
            for n_s in number_of_shots:
                for n_a in number_of_attempts:
                    for a in agentic:
                        ehrm.config(
                            number_of_attempts=n_a,
                            number_of_shots=n_s,
                            agentic=a
                        )

                        ehrm.set_task(
                            task='get_generic_name',
                            possible_shots=possible_shots
                        )

                        preds = ehrm.predict(data[drugn])

                        acc = ehrm.evaluate(data[gener], preds)



In [28]:
model_id

['gpt-3.5-turbo',
 'gpt-4o',
 'meta.llama3-70b-instruct-v1:0',
 'anthropic.claude-v2:1',
 'mistral.mixtral-8x7b-instruct-v0:1']

In [19]:
Engine

{'model_id': ['gpt-3.5-turbo',
  'gpt-4o',
  'meta.llama3-70b-instruct-v1:0',
  'anthropic.claude-v2:1',
  'mistral.mixtral-8x7b-instruct-v0:1'],
 'temperature': [0, 0.1, 0.2, 0.5, 1]}

In [20]:
len(Engine['model_id'])

5

In [ ]:
ehrm = EHRmonize(model_id=Engine['model_id'][0], temperature=Engine['temperature'][0])

In [10]:
function = getattr(ehrm, task_function)

In [ ]:
function()

In [ ]:
for m in model_id:
    for t in temperature:
        ehrm = EHRmonize(model_id=m, temperature=t)
        function = getattr(ehrm, task_function)
        function()